# 04- Structurer la sortie des LLMs
Dans cet exercice, nous allons nous concentrer sur la structuration des réponses des modèles de langage. Jusqu’ici, nous avons vu comment interagir avec un LLM, utiliser des prompts simples et introduire le raisonnement étape par étape avec les Chain of Thought. Cependant, les modèles renvoient souvent des réponses sous forme de texte libre, ce qui peut être difficile à exploiter de manière fiable dans une application.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [ ]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')

## Configuration du llm sur dspy

In [ ]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

En réalité, l’utilisation classique question -> answer n’est qu’un alias simplifié pour créer une interaction avec un LLM : derrière ce raccourci, il est possible de définir des signatures plus complètes basées sur des classes Python. Cela signifie que vous pouvez structurer vos prompts, vos entrées et vos sorties en utilisant des classes qui décrivent explicitement les types et les champs attendus, plutôt que de vous limiter à un texte libre. Cette approche rend vos appels plus robustes, plus lisibles et plus facilement exploitables par votre code, tout en conservant la flexibilité de générer des réponses riches depuis le modèle de langage.

<ins>**Exercice:**</ins> Modifier la signature et l'exécution pour extraire à partir du contenu de l'article `article_content` une liste de plat avec pour chacun le nom, la difficulté(Débutant, Intermédiaire ou Avancé), le temps de préparation et le temps de cuisson

##  Lecture de l'article

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../utils").resolve()))
from file_reader import read_file

article_content = read_file("../../assets/articles/002-article.md")

## Créer une signature

In [ ]:
from enum import Enum
from pydantic import BaseModel, Field

class ExpertCulinaire(dspy.Signature):
    question: str = dspy.InputField(desc="Question posée")
    reponse: str = dspy.OutputField(desc="Réponse à la question")

## Exécuter la prédiction

In [ ]:
from rich import print

assistant = dspy.Predict(ExpertCulinaire)
response = assistant(question="Combien de planètes dans le système solaire ?")

print(response.reponse)
dspy.inspect_history()
print(lm.history)